In [ ]:
# obvykly uvodny tanec
import pandas as pd
import plotly.express as px
import numpy as np  # potrebujeme pre histogram

In [ ]:
# nacitame vycistenu frejmu, stlpce, co pouzijeme v tomto NB, 
cols = ['distance', 'rtime']
df_dt = pd.read_parquet('data/nyc_taxi310k.parq', columns=cols)
df_dt.columns

### V tomto NB nás budú zaujímať časy jázd a prejazdené vzdialenosti

In [ ]:
# predbezny prieskum -rozsahy
for col in df_dt.columns:
    print(col, df_dt[col].min(), df_dt[col].max())

In [ ]:
# ako vela je dlhych a dlhotrvajucich jazd? staci pocet
sd = df_dt[df_dt['distance'] > 15] # skusit 20, ...
st = df_dt[df_dt['rtime'] > 60] # 120, 60, ...
print(sd, st)

In [ ]:
# na zaciatok pre vzd. horna hranica 10, pre casy 30
# motivacia - kvantil - aka cast z 1 je nalavo od neho
# df_dt['distance'].quantile(0.97)
# df_dt['rtime'].quantile(0.97)
# pochopit, co to vrati
yt, xt = np.histogram(df_dt['rtime'], bins=20, range=(0, 40))
print(yt, xt)

In [ ]:
# jedna funkcia pre kreslenie staci - aj s np.histogram vnutri, potom dve funkcie pre vzdial. a casy
def plot_histo(col, rozsah, nbins=20, quant=50):
    xlabel = 'Vzdialenosť (míle)' if col == 'distance' else 'Čas jazdy'
    y, x = np.histogram(df_dt[col], range=rozsah, bins=nbins)
    x = (x[0:-1] + x[1:]) / 2  # stredy intervalov, nie konce
    df_hist = pd.DataFrame({'x': x, 'y': y})  # pomocna frejma
    rt_plot = px.bar(df_hist, x='x', y='y', barmode='group', 
                     labels={'x': xlabel, 'y': 'početnosť', 'variable': 'hodnota'}, width=900, height=350)
    rt_plot.add_vline(df_dt[col].quantile(quant / 100), line={'color':'red'})
    return rt_plot

def histo_dists(bins):
    return plot_histo('distance', (0, 8), bins)

def histo_times(bins):
    return plot_histo('rtime', (0, 35), bins)

In [ ]:
plot_histo('rtime', (0, 40), 40) # skusat 'distance', rozne nbins aj hornu hranicu

In [ ]:
plot_histo('distance', (0, 8), 80)
# preco sa pri niektorom pocte binov histogram pre vzdialenosti 'rozpada' ?
# df.filter(pl.col('distance') == pl.col('distance').round(1))

### Čo hovoria tie grafy? Že je to zrozumiteľnejšie, ako čísla vo frejmoch?

#### Pridáme do `datafunkcie.py` funkcie `plot_histo`, `histo_times`, `histo_dists`
#### pre vyrobu datafrejmy s časmi jázd a vzdialenosťami 